In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
import zipfile
from datetime import datetime
import seaborn as sns

In [ ]:
## TXF
data = pd.read_csv('C:/Users/ASUS/Desktop/TXF1-分鐘-成交價.txt')
data.head()

In [ ]:
## TotalVolume改Volume
data = data.rename(columns = dict(zip(data.columns, ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])))
## 將index改為時間
data.Date = pd.to_datetime(data.Date)
data.Time = pd.to_timedelta(data.Time)
data = data.set_index(data.Date + data.Time).drop(columns = ['Date','Time'])

## 用map抓時間資料
data['Year'] = data.index.map(lambda x: x.year)
data['Month'] = data.index.map(lambda x: x.month)
data['Weekday'] = data.index.map(lambda x: x.weekday)+1
data['Hour'] = data.index.map(lambda x: x.hour)
data.head()

In [ ]:
## FOI
FOI = pd.read_csv('C:/Users/ASUS/Desktop/Foreign_OI.txt')
FOI.head()

In [ ]:
## 將index改為時間
FOI.index = pd.to_datetime(FOI.Date)
FOI = FOI.drop(columns = ['Unnamed: 0','Date'])
FOI

In [ ]:
#data.columns  ### ['Open', 'High', 'Low', 'Close', 'Volume', 'Year', 'Month', 'Weekday','Hour']
## 把data轉成日K
data = data.drop(columns = ['Year', 'Month', 'Weekday','Hour'])
data = data.resample('1d').agg(dict(zip(data.columns, ['first','max','min','last','sum'])))
data

In [ ]:
## 再把日K資料和外資未平倉量資料合併
df = pd.concat([data.Close, FOI.Foreign_OI], axis = 1, join = 'inner')
df

In [ ]:
## 高原圖
# Make the plot
x_sma = []
y_lma = []
z_RonMDD = []

## i短天期範圍，j長天期範圍
for i in range(5,45,5):
    for j in range(60,125,5):
        
        N1 = i
        N2 = j
        df['SMA'] = df.Foreign_OI.rolling(N1).mean().dropna()
        df['LMA'] = df.Foreign_OI.rolling(N2).mean().dropna()
        
        signal = np.where((df.SMA >= df.LMA), 1, -1)
        signal_df = pd.Series(signal, index = df.index)
        
        return_df = pd.concat([df.Close.pct_change(), signal_df.shift(2)], axis = 1).dropna()
        return_df.columns = ['d_return','position']
        return_df['return'] = return_df.d_return * return_df.position
        
        
        MDD_series = return_df['return'].dropna().cumsum().cummax()-return_df['return'].dropna().cumsum()
        MDD = max(MDD_series)
        Return_on_MDD = return_df['return'].dropna().cumsum()[-1] / MDD

        x_sma.append(i)
        y_lma.append(j)
        z_RonMDD.append(Return_on_MDD)

In [ ]:
xyz = {"sma" : x_sma,
       "lma" : y_lma,
       "Return/MDD" : z_RonMDD}
xyz_data = DataFrame(xyz)
xyz_data

In [ ]:
xyz_data_wide = xyz_data.pivot_table( index='sma', columns='lma', values='Return/MDD')
sns.heatmap(xyz_data_wide)
plt.show()